In [ ]:
import sklearn
from scipy.ndimage import label
from sklearn.datasets import make_circles
from sympy import unpolarify
from sympy.printing.pytorch import torch



In [ ]:
nsamples = 1000
x,y = make_circles(n_samples=nsamples,noise=0.03,random_state=42)


In [ ]:
len(x),len(y)

In [ ]:
import pandas as pd

In [ ]:
circles = pd.DataFrame({'x1':x[:,0],'x2':x[:,1],"label":y})
circles.head()

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x=x[:,0],y=x[:,1],c=y,cmap=plt.cm.RdYlBu)

In [ ]:
x.shape,y.shape

In [ ]:
xsample = x[0]
ysample = y[0]
xsample.shape,ysample.shape

In [ ]:
type(x)

In [ ]:
import torch
from torch import nn, inference_mode

In [ ]:
x = torch.from_numpy(x).type(torch.float)
y=torch.from_numpy(y).type(torch.float)

In [ ]:
type(x)
x.shape,y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

In [ ]:
#accessing GPU (mps) for training
device = "mps" if torch.backends.mps.is_available() else "cpu"
device

subclass -> `nn.module` ,almost all models in PY ssubclassess `nn.Module` <br>
create 2 `nn.Linear` layers that are capable of handling shapes of our data <br>
define a `forward()` method that outlines forward pass <br>
instantiate our model and send it o target device (mps)

In [ ]:
class circlemodelv0(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=2,out_features=5)
        self.layer2 = nn.Linear(in_features=5,out_features=1)

    def forward(self,x):
        return self.layer2(self.layer1(x))

model_0 = circlemodelv0().to(device)
model_0


In [ ]:
next(model_0.parameters())

In [ ]:
model_0 = nn.Sequential(nn.Linear(in_features=2,out_features=5),nn.Linear(in_features=5,out_features=1)).to(device)
model_0

In [ ]:
model_0.state_dict()

In [ ]:
xtest[:10],ytest[:10]

In [ ]:
untrainedpred = model_0(xtest.to(device))

In [ ]:
untrainedpred.shape

In [ ]:
lossfn = nn.BCEWithLogitsLoss()

In [ ]:
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.1)

In [ ]:
def accuracyfn(ytrue,ypred):
    corret = torch.eq(ytrue,ypred).sum().item()
    acc = (corret/len(ytrue))*100
    return acc

In [ ]:
model_0.eval()
with torch.inference_mode():
    ylogits = model_0(xtest.to(device))[:5]
ylogits

In [ ]:
#using sigmoid to convert logits into prediction probability
ypredprob = torch.sigmoid(ylogits)
ypredprob

In [ ]:
#pred prob into tensors
ypred = torch.round(ypredprob)
ypredlables = torch.round(torch.sigmoid(model_0(xtest.to(device))[:5]))

print(torch.eq(ypred.squeeze(),ypredlables.squeeze()))

ypred.squeeze()

In [ ]:
ytest[:5]

In [ ]:
torch.manual_seed(42)
torch.mps.manual_seed(42)
epochs = 100

xtrain,ytrain,xtest,ytest = xtrain.to(device),ytrain.to(device),xtest.to(device),ytest.to(device)

for epoch in range(epochs):
    #training
    model_0.train()
    #forward pass
    ylogits = model_0(xtrain).squeeze()
    ypredprob = torch.sigmoid(ylogits)
    ypred = torch.round(ypredprob)
    #loss and optimize
    loss = lossfn(ylogits,ytrain)#BCEwithloh=gitsloss needs logt=its as input,BCEwloss needs pred prob as inputs
    acc = accuracyfn(ytrain,ypred)
    optimizer.zero_grad()
    #loss backward
    loss.backward()
    optimizer.step()
    #evaluate model ,testing
    model_0.eval()
    with torch.inference_mode():
        testlogits = model_0(xtest).squeeze()
        testpredprob = torch.sigmoid(testlogits)
        testpred = torch.round(testpredprob)
        testloss = lossfn(testlogits,ytest)
        testacc = accuracyfn(ytest,testpred)
    if epoch % 10 == 0:
        print(f'epoch: {epoch}, loss: {loss}, acc: {acc} , testloss: {testloss}, testacc: {testacc}')


`plot_decision_boundary`

In [ ]:
import requests
from pathlib import Path

In [ ]:
#downloading any required files from net
if Path("helper_functions.py").is_file():
    print("exists")
else:
    print("downloading")
    request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
    with open("helper_functions.py","wb") as f:
        f.write(request.content)

from helper_functions import plot_predictions,plot_decision_boundary

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("training data")
plot_decision_boundary(model_0,xtrain,ytrain)
plt.subplot(1,2,2)
plt.title("testing data")
plot_decision_boundary(model_0,xtest,ytest)

# improving the model
1. add more layers
2. add more hidden units
3. fit for longer
4. change activation fn
5. chnage lr rate
6. change loss fn

In [ ]:
model_0.state_dict()

In [ ]:
class circlemodelv1(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=2,out_features=10)
        self.layer2 = nn.Linear(in_features=10,out_features=10)
        self.layer3 = nn.Linear(in_features=10,out_features=1)

    def forward(self,x):
        z = self.layer1(x)
        z = self.layer2(z)
        z = self.layer3(z)
        return z

model_1 = circlemodelv1().to(device)
model_1



In [ ]:
model_1.state_dict()

In [ ]:
lossfn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.1)

In [ ]:
torch.manual_seed(42)
torch.mps.manual_seed(42)

epochs = 1000
xtrain,ytrain,xtest,ytest = xtrain.to(device),ytrain.to(device),xtest.to(device),ytest.to(device)
for epoch in range(epochs):
    model_1.train()
    ylogits = model_1(xtrain).squeeze()
    ypredprob = torch.sigmoid(ylogits)
    ypred = torch.round(ypredprob)
    loss = lossfn(ylogits,ytrain)
    acc = accuracyfn(ytrain,ypred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model_1.eval()
    with torch.inference_mode():
        testlogits = model_1(xtest).squeeze()
        testpredprob = torch.sigmoid(testlogits)
        testpred = torch.round(testpredprob)
        testloss = lossfn(testlogits,ytest)
        testacc = accuracyfn(ytest,testpred)
    if epoch % 10 == 0:
        print(f'loss: {loss}, acc: {acc}, testloss: {testloss}, testacc: {testacc:.2f}%')

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("training data")
plot_decision_boundary(model_1,xtrain,ytrain)
plt.subplot(1,2,2)
plt.title("testing data")
plot_decision_boundary(model_1,xtest,ytest)

In [ ]:
weight = 0.7
bias = 0.3

start =0
end =1
step = 0.01

xreg = torch.arange(start,end,step).unsqueeze(dim=1)
yreg = weight * xreg + bias


In [ ]:
trainplit = int(0.8 *len(xreg))
xtrain,ytrain = xreg[:trainplit],yreg[:trainplit]
xtest,ytest = yreg[trainplit:],yreg[trainplit:]
len(xtrain),len(xtest),len(ytrain),len(ytest)

In [ ]:
plot_predictions(xtrain,ytrain,xtest,ytest)

In [ ]:
#adjusting model_1 to fit for straight line
model_2 = nn.Sequential(nn.Linear(in_features=1,out_features=10),
                        nn.Linear(in_features=10,out_features=10),
                        nn.Linear(in_features=10,out_features=1)).to(device)
model_2

In [ ]:
lossfn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model_2.parameters(), lr=0.01)

In [ ]:
torch.manual_seed(42)
torch.mps.manual_seed(42)
epochs = 1000

xtrain,ytrain,xtest,ytest = xtrain.to(device),ytrain.to(device),xtest.to(device),ytest.to(device)

for epoch in range(epochs):
    model_2.train()
    ypred = model_2(xtrain)
    loss = lossfn(ypred,ytrain)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model_2.eval()
    with torch.inference_mode():
        testpred = model_2(xtest)
        testloss = lossfn(testpred,ytest)
    if epoch %100==0:
        print(f'loss: {loss}, testloss: {testloss:.2f}%')

In [ ]:
model_2.state_dict()

In [ ]:
model_2.eval()
with torch.inference_mode():
    testpred = model_2(xtest)

plot_predictions(xtrain.cpu(),ytrain.cpu(),xtest.cpu(),ytest.cpu(),predictions=testpred.cpu())

Non-linearity

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles

nsamples = 1000

x,y = make_circles(n_samples=nsamples,noise=0.03,random_state=42)
plt.scatter(x[:,0],x[:,1], c=y,cmap=plt.cm.RdYlBu)

In [ ]:
import torch
from sklearn.model_selection import train_test_split

x =torch.from_numpy(x).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

xtrain,ytrain,xtest,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from torch import nn

class circlemodelv2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1  = nn.Linear(in_features=2,out_features=10)
        self.layer2 = nn.Linear(in_features=10,out_features=10)
        self.layer3 = nn.Linear(in_features=10,out_features=1)
        self.relu = nn.ReLU() #relu is a activation funtion

    def forward(self,x):
       return self.layer3(self.relu(self.layer2(self.relu(self.layer1(x)))))

model_3 = circlemodelv2().to(device)
model_3

In [ ]:
lossfn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_3.parameters(), lr=0.1)

In [ ]:
ytrain

In [ ]:
torch.manual_seed(42)
torch.mps.manual_seed(42)

epochs = 1000
xtrain,ytrain,xtest,ytest = xtrain.to(device),ytrain.to(device),xtest.to(device),ytest.to(device)
for epoch in range(epochs):
    model_3.train()
    y_logits = model_3(xtrain).squeeze()
    ypredprob = torch.sigmoid(y_logits)
    ypred = torch.round(ypredprob)
    loss = lossfn(y_logits,ytrain)
    acc = accuracyfn(ytrain,ypred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model_1.eval()
    with torch.inference_mode():
        testlogits = model_3(xtest).squeeze()
        testpredprob = torch.sigmoid(testlogits)
        testpred = torch.round(testpredprob)
        testloss = lossfn(testlogits,ytest)
        testacc = accuracyfn(ytest,testpred)
    if epoch % 10 == 0:
        print(f'loss: {loss}, acc: {acc}, testloss: {testloss}, testacc: {testacc:.2f}%')

In [ ]:
model_3.eval()
with torch.inference_mode():
    testlogits = model_3(xtest).squeeze()
    testpredprob = torch.sigmoid(testlogits)
    testpred = torch.round(testpredprob)

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("training data")
plot_decision_boundary(model_1,xtrain,ytrain) # linear model
plt.subplot(1,2,2)
plt.title("testing data")
plot_decision_boundary(model_3,xtest,ytest) # non linear modelS

replicting non linear activation funtions

In [ ]:
A = torch.arange(-10,10,1,dtype=torch.float)
A.dtype

In [ ]:
A

In [ ]:
plt.plot(A)

In [ ]:
plt.plot(torch.relu(A))

In [ ]:
def relu(x)->torch.Tensor:
    return torch.maximum(torch.tensor(0),x)

In [ ]:
relu(A)

Multiclasss classification

In [ ]:
import torch
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

numclass = 4
numfeatures = 2
randomseed = 42

xblob , yblob = make_blobs(n_samples=1000,n_features=numfeatures,random_state=randomseed,centers=numclass,cluster_std=1.5)

xblob = torch.from_numpy(xblob).type(torch.float)
yblob = torch.from_numpy(yblob).type(torch.float)

In [ ]:
xblobtrain,xtestblob,yblobtrain,ytestblob = train_test_split(xblob,yblob,test_size=0.2,random_state=42)

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(xblob[:,0],xblob[:,1],c=yblob,cmap=plt.cm.RdYlBu)

In [ ]:
#multiclass classification model
class blobModel(nn.Module):
    def __init__(self,input_features,output_features,hidden_units = 8):
        super().__init__()
        self.linearlayerstack = nn.Sequential(nn.Linear(in_features=input_features,out_features=hidden_units),
                                              nn.ReLU(),
                                              nn.Linear(in_features=hidden_units,out_features=hidden_units),
                                              nn.ReLU(),
                                              nn.Linear(in_features=hidden_units,out_features=output_features))
    def forward(self,x):
        return self.linearlayerstack(x)

model_4 = blobModel(2,4,8).to(device)
model_4

In [ ]:
xtrain.shape,ytrain.shape

In [ ]:
lossfuntion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_4.parameters(), lr=0.1)

In [ ]:
model_4.eval()
with torch.inference_mode():
    ylogits = model_4(xtestblob.to(device))

In [ ]:
ypredprob = torch.softmax(ylogits,dim=1)

In [ ]:
ypred = torch.argmax(ypredprob,dim=1)
ypred

In [ ]:
#training and testing loop for multiclass model
torch.manual_seed(42)
torch.mps.manual_seed(42)

epochs = 100

xblobtrain,yblobtrain,xtestblob,ytestblob = xblobtrain.to(device),yblobtrain.to(device),xtestblob.to(device),ytestblob.to(device)

for epoch in range(epochs):
    model_4.train()
    ylogits = model_4(xblobtrain)
    ypred = torch.softmax(ylogits,dim=1).argmax(dim=1)
    loss = lossfuntion(ylogits,yblobtrain)
    acc = accuracyfn(yblobtrain,ypred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model_4.eval()
    with torch.inference_mode():
        testlogits = model_4(xtestblob)
        testpred = torch.softmax(testlogits,dim=1).argmax(dim=1)
        testloss = lossfuntion(testlogits,ytestblob)
    if epoch % 10 == 0:
        print(f'loss : {loss}, acc: {acc:.2f}% ')

In [ ]:
model_4.eval()
with torch.inference_mode():
    ylogits = model_4(xtestblob)
ylogits

In [ ]:
ypredprod = torch.softmax(ylogits,dim=1)
ypred = torch.argmax(ypredprob,dim=1)

In [ ]:
ypred[:10]

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("training data")
plot_decision_boundary(model_4,xtestblob,ytestblob)
plt.subplot(1,2,2)
plt.title("testing data")
plot_decision_boundary(model_4,xtestblob,ytestblob)

In [ ]:
from torchmetrics import Accuracy


In [ ]:
torchmetricacc = Accuracy(task="multiclass",num_classes=numclass).to(device)
torchmetricacc(ypred,ytestblob)